# Simple Trend Following Strategy Example

This notebook demonstrates a basic trend following strategy using moving averages.

In [ ]:
# Import required libraries
from crypto_backtest import run_backtest, load_data
from crypto_backtest.features import sma, ema, rsi, atr
import pandas as pd
import numpy as np

In [ ]:
# Configuration
SYMBOL = 'BTC/USDT'
EXCHANGE = 'binance'
TIMEFRAME = '1h'
START_DATE = '2024-01-01'
END_DATE = '2024-06-30'
INITIAL_CAPITAL = 10000

In [ ]:
# Load data
print("Loading data...")
data = load_data([SYMBOL], EXCHANGE, TIMEFRAME, START_DATE, END_DATE)
print(f"Loaded {len(data[SYMBOL]['ohlcv'])} bars")

In [ ]:
# Define trend following strategy
def trend_following_strategy(data, position, timestamp, **params):
    """
    Simple trend following strategy using dual moving averages
    """
    # Get price data
    ohlcv = data[SYMBOL]['ohlcv']
    if len(ohlcv) < params['slow_ma']:
        return []  # Not enough data
    
    # Calculate indicators
    close = ohlcv['close']
    fast_ma = sma(close, params['fast_ma'])
    slow_ma = sma(close, params['slow_ma'])
    rsi_val = rsi(close, params['rsi_period'])
    
    # Get current values
    current_fast = fast_ma.iloc[-1]
    current_slow = slow_ma.iloc[-1]
    current_rsi = rsi_val.iloc[-1]
    current_pos = position.get(SYMBOL, 0)
    
    # Generate signals
    if current_fast > current_slow and current_rsi < params['rsi_overbought']:
        if current_pos == 0:
            # Enter long position
            return [{'symbol': SYMBOL, 'side': 'buy', 'size': params['position_size']}]
    
    elif current_fast < current_slow or current_rsi > params['rsi_overbought']:
        if current_pos > 0:
            # Exit long position
            return [{'symbol': SYMBOL, 'side': 'sell', 'size': current_pos}]
    
    return []

In [ ]:
# Define strategy parameters
params = {
    'fast_ma': 20,
    'slow_ma': 50,
    'rsi_period': 14,
    'rsi_overbought': 70,
    'position_size': 0.1  # 0.1 BTC per trade
}

In [ ]:
# Run backtest
print("Running backtest...")
results = run_backtest(
    data=data,
    strategy=trend_following_strategy,
    initial_capital=INITIAL_CAPITAL,
    params=params,
    commission=0.001  # 0.1% commission
)

In [ ]:
# Display results summary
print(results.summary())

In [ ]:
# Plot results
results.plot()

## Parameter Optimization

Let's optimize the strategy parameters to find the best combination.

In [ ]:
from crypto_backtest import optimize_strategy

# Define parameter search space
param_space = {
    'fast_ma': (10, 30),
    'slow_ma': (40, 100),
    'rsi_period': (10, 20),
    'rsi_overbought': (65, 80),
    'position_size': (0.05, 0.2)
}

# Run optimization
print("Running parameter optimization...")
opt_results = optimize_strategy(
    data=data,
    strategy=trend_following_strategy,
    param_space=param_space,
    metric='sharpe_ratio',
    n_trials=50,
    initial_capital=INITIAL_CAPITAL
)

print(f"\nBest parameters: {opt_results['best_params']}")
print(f"Best Sharpe ratio: {opt_results['best_value']:.2f}")

In [ ]:
# Display optimized results
print(opt_results['final_results'].summary())
opt_results['final_results'].plot()